# Project progress - 15/5/19

we had a big problem with the calculation of the PN (location dependent) part of the model. 
To so, it was requiered to loop on a very big fasta file, and to do calculations on every sequence in the fasta file sepratly  and it took a lot of running time.
so we devised another method. Although technically the time complexity hasn't changed with this method, it uses matrix multipction and other numpy method instead of looping, and so it reduced the running time. 

The method:
1. we will loop on the fasta file, but in every iteration, we will read a lot of lines, 100000, for example, into a numpy array. This way we will reduce significantly the number of iteration in the loop.
2. we will filter the lines containing only sequence from this array, and we will get matrix filled only with sequence strings.


Now, what we can do with this matrix? how we will get the sequence matrix from every line? 
In previous method, we created a sequence matrix from every sequence, and summed them togther. It also required a lot of runing time. 
In our current method, we dont need to work on every sequence seperately! 
In the previous method, to devise a sequence matrix for every sequence, we looped on the sequence, took every i'th k-mer, converted it to its base-4 represntation, lets call it m, and then initinialized it the [i,m] cell in the sequence matrix with 1. 
for example: 

In [20]:
import numpy as np
import string
translation = string.maketrans("ACGT", "0123")

seq = "ACTG"
seq = string.translate(seq,translation)
markov_order = 1



seq_mat = np.zeros(((len(seq) - markov_order) , np.power(4, markov_order+1)))

print "number representing k-mers:\n"
for i in range(0, len(seq)-markov_order):
    current_num = 0
    for j in range(0, markov_order+1):
        current_num = current_num + np.power(4,markov_order -j)*int(seq[i+j])
    print current_num 
    seq_mat[i, current_num] = 1

print seq_mat

number representing k-mers:

1
7
14
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]


In [19]:
seq_len = len(seq)
summing_matrix = np.zeros((seq_len, seq_len - markov_order))


for i in range(0,markov_order+1):
    np.fill_diagonal(summing_matrix[::][i:], np.power(4,markov_order- i))

print summing_matrix

[[4. 0. 0.]
 [1. 4. 0.]
 [0. 1. 4.]
 [0. 0. 1.]]


In [18]:
seq = string.translate(seq,translation)
a =  np.array(map(int, [char for char in seq]))  
k_mers = np.matmul(a ,summing_matrix )
print k_mers

[ 1.  7. 14.]


In [17]:
seq1 = "ACGT"
seq2 = "AACG"
seq3= "ACGA"
seq4 = "CGTA"
seq_list = [seq1, seq2,seq3,seq4]
def to_array(seq):
    seq = string.translate(seq,translation)
    a =  np.array(map(int, [char for char in seq]))  
    
    return a
seqs_array = map(to_array,seq_list)
print np.array(seqs_array)

[[0 1 2 3]
 [0 0 1 2]
 [0 1 2 0]
 [1 2 3 0]]


In [16]:
print np.matmul(seqs_array, summing_matrix)

[[ 1.  6. 11.]
 [ 0.  1.  6.]
 [ 1.  6.  8.]
 [ 6. 11. 12.]]


so we got all the kmers base 4 represntation for every sequence just by multiplying matrices. But how does it help us to devise the sequence matrix? you can say that we need to iterate on every row and get the squence matrix, but this wouldn't be very effiecient. 
so what will we do? 
in out method, we dont need to create any sequence matrix. consider this - every i'th coloumn in the matrix that we created in a list of all the different kmers starting from the i'th position on the sequences. so in our little example,  the first coloumn is [1,0,1,6]. Those are base 4 represenations of the kmers AC,AA,AC,CG respectivly. and you can see that those are the kmers on each sequence from the sequences that we showed. 
so in this example, we can make a one dimentional array for every coloumn, and the i'th cell will be initialized with the number of kmers the has represntation in base 4 that is i. 




In [15]:
protein_kmers = np.matmul(seqs_array, summing_matrix)
N = np.power(4, markov_order+1 )

protein_kmers = np.transpose(protein_kmers)
_id = protein_kmers + (N * np.arange(protein_kmers.shape[0], dtype = np.int64))[:,None]

kmers_counts = np.bincount((_id.ravel()).astype(np.int64),minlength=N*protein_kmers.shape[0]).reshape(-1,N)

print kmers_counts

[[1 2 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 2 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 1 0 1 0 0 1 1 0 0 0]]
